In [7]:
'''
Created on Dec 18 2023
@author: tim@footprintlab.io


'''
from copy import copy
from dataclasses import dataclass
from typing import Optional
import pandas as pd
import numpy as np
import csv
import os


# integration notes
# have incorporated
#  ioanalysis(transaction_table,total_output,final_demand,M_) -> technical_coefficient,direct_intensity_multiplier,total_intensity_multiplier


# Summary of functions

# single_region_su_no_imports(T, data_V, data_Y, data_TQ)
    # V, Y, X, M

# single_region_su_imports_as_primary_inputs(T, imports, data_V, data_Y, data_TQ, xt) -> #  V, Y, X, M:
    

# Define regional and sectoral extents
countries = 2 #164
sectors = 4 # 120


#### SINGLE SU I as in ECE  ###############################
#     IELAB Portal standard outputs with imports          #
#                                                         #
#                                                         #
# Single region SU table with TQ and imports treated as in
# ECE model: imports used to calculate total output, X but
# ROW not included as column, nor Imports as row, in L



# ??? Difference between these
# T  = np.genfromtxt(inputPath + filenameRoot + str(Phase) + 'T-Results' + '.csv', delimiter=",", dtype=float) 
# data_V  = np.genfromtxt(inputPath + filenameRoot + str(Phase) + 'V-Results' + '.csv', delimiter=",", dtype=float) 
# data_Y = np.genfromtxt(inputPath + filenameRoot + str(Phase) + 'Y-Results' + '.csv', delimiter=",", dtype=float)
# data_TQ = np.genfromtxt(inputPath + filenameRoot + str(Phase) + 'TQ-Results' + '.csv', delimiter=",", dtype=float)

import numpy.typing as npt
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subplots

@dataclass
class Parameters:
    transaction_table: npt.NDArray
    primary_inputs: npt.NDArray
    final_demand: npt.NDArray
    satellite_data: npt.NDArray
    num_countries: int
    num_sectors: int
    multiplier:Optional[npt.NDArray] = None
    imports: Optional[npt.NDArray] = None
    
    
    # m is actually chosen from TQ, the satellite
    # cf M = data_TQ #[:,:industry_product_ext]
    
class IOTable:
    """
    Has basic attributes:
    T: Transaction table
    Y: Final demand
    TQ: Satellite data
    """
    def __init__(self,transaction_table, 
                        final_demand, 
                        # satellite_data, 
                        primary_inputs,
                        num_sectors, 
                        num_countries,
                        multipliers:Optional[npt.NDArray] = None,
                        imports:Optional[npt.NDArray] = None):
        
        self.transaction_table = transaction_table
        self.final_demand = final_demand
        # self.satellite_data = satellite_data
        self.primary_inputs = primary_inputs
        self.sectors = sectors
        self.num_sectors = len(self.sectors)
        # self.commodities = commodities
        # self.num_sectors = len(self.sectors)
        self.countries = countries
        self.num_countries = len(self.countries)
        self.multipliers = multipliers
        self.imports = imports
        
        
        # set up indices
        self._commodity_rows = [(c * 2 + 1) * self.num_sectors + sr for sr in range(self.num_sectors) for c in range(self.num_countries)]
        self._commodity_columns = [c * 2 * self.num_sectors + sr for sr in range(self.num_sectors) for c in range(self.num_countries)]
        self._supply_rows = [c * 2 * self.num_sectors + sr for sr in range(0,self.num_sectors) for c in range(0,self.num_countries)]
        self._supply_columns = [(c * 2 +1) * self.num_sectors + sr for sr in range(0,self.num_sectors) for c in range(0,self.num_countries)]
        self._commodity_rows.sort()
        self._commodity_columns.sort()
        self._supply_rows.sort()
        self._supply_columns.sort()
        
        # from method_SingleSUI_NoImports
        # 
        self.total_output = np.sum(self.transaction_table, axis=0, keepdims=True) + np.sum(self.primary_inputs, axis=0, keepdims=True)
        self.technical_coefficient = self.transaction_table/self.total_output
        self.identity = np.identity(self.technical_coefficient.shape[0])
        self.leontief_inverse = np.linalg.inv(self.identity-self.technical_coefficient)
        
        
        self.multipliers = multipliers
        if multipliers is not None:
            
            self.direct_intensity_multipliers = self.multipliers/self.total_output
            self.total_intensity_multipliers = np.transpose(np.dot(self.direct_intensity_multipliers,self.leontief_inverse))
            
  
    
    @classmethod 
    def from_file(cls,path,num_sectors,num_countries):
    # descriptive naming
        pars_from_file = Parameters(
        transaction_table = np.genfromtxt(os.path.join(path ,'T.csv'), delimiter=",", dtype=float),
        primary_inputs = np.genfromtxt(os.path.join(path ,'V.csv'), delimiter=",", dtype=float),
        final_demand = np.genfromtxt(os.path.join(path ,'Y.csv'), delimiter=",", dtype=float),
        satellite_data = np.genfromtxt(os.path.join(path ,'TQ.csv'), delimiter=",", dtype=float),
        num_sectors=num_sectors,
        num_countries=num_countries)
        return cls(pars_from_file)        
        

    # def create_chain(self,max_layers:int)->xr.Dataset:
    #     """Create a structure to house the data"""
    #     # validation!!!
    #     assert self.multipliers is not None
        
    #     # todo coerce functions for column_vector, row_vector
        
    #     extended_table = np.concatenate([self.transaction_table,np.atleast_2d(self.final_demand).T],axis=1)
    #     output = np.sum(extended_table,axis=1)
    #     tech_coef = self.transaction_table/np.atleast_2d(output).T
        
    #     supply_chain_data = np.zeros((*tech_coef.shape,max_layers))
    #     layer_transfer = np.eye(tech_coef.shape[0])
    #     for layer in range(max_layers):
    #         # create the next level of dependency
    #         supply_chain_data[:,:,layer] = np.dot(layer_transfer,tech_coef)
    #         layer_transfer = copy(supply_chain_data[:,:,layer])

    #     chain = xr.Dataset(data_vars={'transfer':(['supplier','user','layer'],
    #                                         supply_chain_data),
    #                                         'direct_intensity':(['supplier'],self.direct_intensity_multipliers),
    #                                         'total_intensity':(['supplier'],self.total_intensity_multipliers),
    #                                         'final_demand':(['supplier'],self.final_demand),
    #                                         'output':(['supplier'],output)}, 
    #                             coords={'supplier':self.sector_list,
    #                                     'user':self.sector_list,
    #                                     'layer':range(max_layers)})
        
        
        
    #     # construct other useful objects for downstream use
    #     direct = chain.output*chain.direct_intensity
    #     direct.name = 'absolute_direct'
    #     chain=chain.merge(direct)
        
    #     total = chain.output*chain.total_intensity
    #     total.name = 'absolute_total'
    #     chain=chain.merge(total)
        
    #     self.chain = chain



    # def _product_layer_decomposition(self):
    #     """Compute the PLD as per Wieland et al 2018
    #     returns a tensor, each column (at a given layer) shows the carbon emissions 
    #     from a supplier-sector (row) embodied in the user-sector (column)."""
    #     #  the emissions at layer $k$ are

    #     # $E_k = \hat{s} A^k \hat{y}$

    #     # where $s$ and $y$ are the diagonalized direct intensities and the diagonalised final demand.
    #     # The first product is just to achieve scaling of the rows which can be done by
    #     # broadcast multiplication. Ditto the latter, which scales the columns.
    #     # xarray provides very nice ways to implement broadcast multiplication:

    #     user_demand = self.final_demand.rename({'supplier':'user'})
    #     specific_intensity = (self.direct_intensity/self.output)
    #     return specific_intensity*self.transfer*user_demand

    # def show_pld(self,user_sector:str):
    #     pld = self._product_layer_decomposition()
    #     _subframe = pld.sel(user=user_sector).to_dataset(name='emissions')
    #     pld_grid = _subframe.to_dataframe().reset_index()
    #     cumulative_emissions = _subframe.cumsum('layer').to_dataframe().reset_index()
            
    #     sp1 = px.bar(pld_grid,
    #         x='layer',y='emissions',color='supplier')
    #     sp2 = px.bar(cumulative_emissions,
    #         x='layer',y='emissions',color='supplier')

    #     fig = subplots.make_subplots(rows=1,cols=2)


    #     for tr in sp1.data:
    #         tr.showlegend=False
    #         fig.add_trace(tr,
    #         row=1,col=1)
    #     for tr in sp2.data:
    #         fig.add_trace(tr,
    #         1,2)
    #     fig.update_xaxes(title_text='Layer')
    #     fig.update_yaxes(title_text='Direct emissions',row=1,col=1)
    #     fig.update_yaxes(title_text='Cumulative emissions',row=1,col=2)
    #     fig.update_layout(barmode='stack',width=850,title=f"Product layer decomposition for {user_sector}")

    #     fig.show()        




# num_countries = 2
# num_sectors = 4
# # a 'sector' here is a member of a list used to describe both industries and products
# # which is generally how it's done in IELab data though not in the SPA toy example ...
# # so, there's a need to generalise the following to allow for SU formatted tables with
# # quadrants of different extents as in the example from Wiedmann (2017)

# #e.g. for GLORIA
# #countries = 164
# #sectors = 120
# # May be needed later for sourcing/ using IELab files
# #Phase = '002'
# #Loop = '059'
# # TQr=TQr[[3168,5943]] .. for GLORIA this further selects the two impact rows with total CO2-eq emissions (from EDGAR and OECD respectively)

# iot = IOTable.from_file('data/tutorial',num_sectors,num_countries)



## Data input

In [8]:
import numpy as np

user_def=True

## Provide 
    
    # 2-region-sut-to-siot
transaction_table = np.genfromtxt('data/tutorial/T.csv', delimiter=",", dtype=float)
primary_inputs = np.genfromtxt('data/tutorial/V.csv', delimiter=",", dtype=float)
final_demand = np.genfromtxt('data/tutorial/Y.csv', delimiter=",", dtype=float)
direct_emissions = np.genfromtxt('data/tutorial/TQ.csv', delimiter=",", dtype=float)

countries = ['Australia','Belize']
commodity_list = ['atoms','bits','cats','dogs']
sector_list = ['eating','farming','golfing','helping']

tutorial_iotable = IOTable(
    transaction_table=transaction_table,
    primary_inputs=primary_inputs,
    final_demand=final_demand,
    multipliers=direct_emissions,
    num_countries=2,
    num_sectors=4
    )

tutorial_iotable.show_pld()

In [39]:
# testing old snippets
T = np.genfromtxt('data/gloria/T.csv', delimiter=",", dtype=float)
print(T.shape)
V = np.genfromtxt('data/gloria/V.csv', delimiter=",", dtype=float)
print(V.shape)
Y = np.genfromtxt('data/gloria/Y.csv', delimiter=",", dtype=float)
print(Y.shape)
TQ = np.genfromtxt('data/gloria/TQ.csv', delimiter=",", dtype=float)
print(TQ.shape)

# these data have some extra bits


(1032, 689)
(5, 689)
(1032, 6)
(176, 689)
[4, 12, 5, 13, 6, 14, 7, 15]
[0, 8, 1, 9, 2, 10, 3, 11]
[0, 8, 1, 9, 2, 10, 3, 11]
[4, 12, 5, 13, 6, 14, 7, 15]


ValueError: shapes (8,689) and (8,689) not aligned: 689 (dim 1) != 8 (dim 0)

In [ ]:

# T = np.genfromtxt('data/gloria/' + '20240110_120secMother_AllCountries_002_T-Results_2021_059_Markup001(full).csv', delimiter=",", dtype=float)
# V = np.genfromtxt('data/gloria/' + '20240110_120secMother_AllCountries_002_V-Results_2021_059_Markup001(full).csv', delimiter=",", dtype=float)
# Y = np.genfromtxt('data/gloria/' + '20240110_120secMother_AllCountries_002_Y-Results_2021_059_Markup001(full).csv', delimiter=",", dtype=float)
# TQ = np.genfromtxt('data/gloria/' + '20231117_120secMother_AllCountries_002_TQ-Results_2021_059_Markup001(full).csv', delimiter=",", dtype=float)

countries = 2
sectors = 4

def make_iot_tables(transaction_table,V,Y,TQ):

    total_output = np.sum(transaction_table, axis=0, keepdims=True) + np.sum(V, axis=0, keepdims=True)
    technical_coefficient = transaction_table/total_output

    #select only Commodity block rows and colunms
    commodity_rows = [(c * 2 + 1) * sectors + sr for sr in list(range(sectors)) for c in list(range(countries))]
    print(commodity_rows)
    commodity_rows.sort()

    commodity_columns = [c * 2 * sectors + sr for sr in list(range(sectors)) for c in list(range(countries))]
    print(commodity_columns)
    commodity_columns.sort()

    B = technical_coefficient.take(commodity_rows, axis=0)

    supply_rows = [c * 2 * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
    print(supply_rows)
    supply_rows.sort()

    supply_columns = [(c * 2 +1) * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
    print(supply_columns)
    supply_columns.sort()

    D = technical_coefficient.take(supply_rows, axis=0)
    
    Aq = np.dot(B,D)

    xt = Aq.shape[1]
    identity = np.identity(xt) 
    # L = np.linalg.inv(identity-Aq)
    # DL =  np.dot(D,L)


    # Xr = total_output.take(commodity_columns) # for industry output X.take(supply_columns)),axis=0)
    # Yr = Y.take(commodity_rows, axis=0)
    # TQr = TQ.take(commodity_columns, axis=1)
    # # TQr=TQr[[3168,5943]] .. for GLORIA this further selects the two impact rows with total CO2-eq emissions (from EDGAR and OECD respectively)
    # Ar = identity - np.linalg.inv(DL)
    # Tr = Ar * Xr
    print('loaded mostly')
    # return Tr,Yr,TQr

Tr,Yr,TQr = make_iot_tables(T,V,Y,TQ)




In [38]:
# -*- coding: utf-8 -*-
"""
Created on June 7 2024
Last edit 7 June 2024
@author: tim@footprintlab.io

Takes Supply Use (SU) formatted input-output tables:
T - Transaction
V - Primary inputs
Y - Final demand
TQ - Satellite data under the T table

returns 'symmetric' (IOT) formatted input-output tables
and selected results from an IO analysis on IOT tables

"""

import numpy as np
import os
import time

'''%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
PATHS AND GLOBALS
'''
print("Setting Paths and Globals")
print("")


DATAPATH =  'data/su_to_iot_tutorial/'
# DATAPATH =  'data/gloria/'

# # Define regional and sectoral extents
# countries = 164
# sectors = 120
# Phase = '002'
# Loop = '059'

start = time.time()

'''%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
DATA
'''
print("Reading in Base Tables")
print("")

T = np.genfromtxt(DATAPATH + 'T.csv', delimiter=",", dtype=float)
print(f"T shape {T.shape}")
V = np.genfromtxt(DATAPATH + 'V.csv', delimiter=",", dtype=float)
print(f"V shape {V.shape}")
Y = np.genfromtxt(DATAPATH + 'Y.csv', delimiter=",", dtype=float)
print(f"Y shape {Y.shape}")
TQ = np.genfromtxt(DATAPATH + 'TQ.csv', delimiter=",", dtype=float)
print(f"TQ shape {TQ.shape}")

countries = 2
sectors = 4


Setting Paths and Globals

Reading in Base Tables

T shape (16, 16)
V shape (16,)
Y shape (16,)
TQ shape (16,)


In [ ]:



X = np.sum(T, axis=0, keepdims=True) + np.sum(V, axis=0, keepdims=True)
print(X.shape)

A = T/X

print("Creating B Table")
print("")
#select only Commodity block rows and colunms
commodity_rows = [(c * 2 + 1) * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
commodity_rows.sort()

commodity_columns = [c * 2 * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
commodity_columns.sort()

B = A.take(commodity_rows, axis=0)
B = B.take(commodity_columns, axis=1)


now = time.time()
print((now-start)/60)

print("Creating D Table")
print("")
#select only Supply block rows and colunms
supply_rows = [c * 2 * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
supply_rows.sort()

supply_columns = [(c * 2 +1) * sectors + sr for sr in list(range(0,sectors)) for c in list(range(0,countries))]
supply_columns.sort()

D = A.take(supply_rows, axis=0)
D = D.take(supply_columns, axis=1)

now = time.time()
# print((now-start)/60)
# print()

Aq = np.dot(B,D)

xt = Aq.shape[1]
I = np.identity(xt) 

L = np.linalg.inv(I-Aq)
DL =  np.dot(D,L)

print("Made DL")
now = time.time()
# print((now-start)/60)
# print()

Xr = X.take(commodity_columns) # for industry output X.take(supply_columns)),axis=0)
Yr = Y.take(commodity_rows, axis=0)
# TQr = TQ.take(commodity_columns, axis=1)
Vr =V[commodity_columns]
TQr =TQ[commodity_columns]
# TQr=TQr[[3168,5943]]

Ar = np.linalg.inv(I-DL)
Tr = Ar * Xr


print("Made T IOT")
print()
print("Total minutes to run =", (time.time()-start)/60)	
print()
print ("----Finished ----")